# Ejercicio 1

In [66]:
from sklearn.datasets import fetch_20newsgroups
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import seaborn as sns

## Importación de los sets de train y test.
Se eliminan los headers y los footers.
Se verifican los tamaños de los sets y las categorías a las que corresponden los artículos.

In [67]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, remove=('headers', 'footers'))
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, remove=('headers', 'footers'))
print('Length train: '+ str(len(twenty_train.data)))
print('Length test: '+ str(len(twenty_test.data)))

print(twenty_train.target_names)

Length train: 11314
Length test: 7532
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [68]:
# A fin de analizar el formato de los artículos, se imprimen los primeros 5.
for i in range(5):
    print('NEW DATA')
    print(twenty_train.data[i])

NEW DATA
I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.
NEW DATA
A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't ans

## Preprocesamiento.

Antes de alimentar al modelo con los textos, es necesario transformarlos a fin de aumentar la consistencia de la métrica a obtener. Para ello se siguen una serie de pasos que se explican a continuación,.

### Tokenización
La tokenización es el proceso por el cual una gran cantidad de texto se divide en partes más pequeñas llamadas tokens. Los tokens serán utilizados para posteriormente procesar individualmente cada palabra del texto.
### Lematización
La lematización tiene en cuenta el análisis morfológico de las palabras para transformarlas. Distintas palabras que corresponden a un mismo concepto pero varían levemente -o no varían- en su significado son llevadas a una forma en común que las representa a todas. En general esta forma coincide con el formato de las palabras que se encuentran en el diccionario. Por ejemplo, los verbos son transformados en sus infinitivos y los plurales en singulares.
### Stopwords
Las "stopwords" son las palabras comunes más comunes en un idioma. Por esta razón, tenerlas en cuenta para el modelo que se desea implementar no aporta información relevante en cuanto a la categorización de los textos e implica mayor tiempo de procesamiento. Como ejemplo las palabras "a", "de", "y", "el" y "o" son consideradas stopwords en el idioma español.
### Estemizado
El estemizado reduce las palabras a su raíz y es utilizado en el preprocesamiento del texto con el objetivo de unificar términos cuyos sufijos varían pero pertenecen a las mismas clases. Por ejemplo, las palabras "niños" y "niñez" podrían ser reducidas mediante esta técnica a la raíz "niñ".
### Filtrado de no alfabéticos.
Una vez aplicadas las técnicas mencionadas anteriormente, es probable que el vocabulario siga contando con tokens que no sean palabras. Estos tokens, como signos de puntuación o números, no aportan información útil al procesamiento por lo tanto deben ser eliminados para mejorar el rendimiento del modelo.

### NLTK
La librería NLTK (Natural Language Toolkit) contiene herramientas y paquetes con funcionalidades relacionadas al procesamiento estadístico del lenguaje natural. Entre estas herramientas se encuentran las explicadas anteriormente (lematización, estemizado, etc.).

#### Selección del Stemmer
Los tres principales algoritmos de stemming que se utilizan son: Porter Stemmer, Snowball Stemmer y Lancaster Stemmer.  
El algoritmo de Porter es el más antiguo de los tres algoritmos mencionados y es el menos "agresivo". A su vez, implica un alto costo en términos de tiempo de procesamiento. Por su parte, el algoritmo Snowball presenta algunas mejoras con respecto al de Porter y en cuanto a tiempo de procesamiento es algo mas rápido. Por último, el algoritmo de Lancaster es el más rapido de los tres pero el resultado del lematizado es poco intuitivo y de difícil análisis.  
Analizando los beneficios y contras de cada lematizador, decidimos utilizar el ***Snowball Stemmer***, provisto por NLTK. 

#### Selección del Lematizador
***Wordnet*** es una base de datos léxica que proporciona relaciones semánticas entre sus palabras. Wordnet agrupa distintos elementos de datos que son semánticamente equivalentes en "synsets". Es una de las primeras y más utilizadas técnicas de lematización y se encuentra disponible con NLTK. Por estas razones, decidimos utilizar dicho lematizador.  
Sin embargo, este lematizador trata por defecto a todas las palabras como si fueran sustantivos. Así, muchos verbos se mantienen iguales luego de la lematización. Para solucionar este problema, es necesario utilizar etiquetas "***POS***" (Part Of Speech). Por cada palabra, se llama al lematizador con una etiqueta que define su tipo (adjetivo, sustantivo, verbo, etc.). 

 

In [69]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') #For POS tags

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pabli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pabli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pabli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pabli\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [70]:
# Inicialización del lematizador y del stemmer.
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

In [71]:
#Traduce etiquetas POS de la función nltk.pos_tag en etiquetas que acepta WordNetLemmatizer.
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [72]:
def pre_process_data(data_in):
    filtered_arts = list()
    aux_porc = -1
    for i in range(len(data_in)):
        ########## PRINT PROGRESS ############
        porc = int((i/len(data_in))*100)
        if (porc!=aux_porc):
            print(str("\rProcesado: " + str(porc) + "%"), end="")
            porc = aux_porc
        ############## TOKENIZE ################
        tokenized = word_tokenize(data_in[i])
        ############## LEMMATIZE ################
        pos_tagged = nltk.pos_tag(tokenized)
        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
        lemmatized = list()
        for word, tag in wordnet_tagged:
            if tag is None:
                lemmatized.append(word)
            else:        
                lemmatized.append(lemmatizer.lemmatize(word, tag))
        ############## STOP WORDS ################
        stop = [lem for lem in lemmatized if lem not in stopwords.words('english')]
        ############## STEMMER ################
        stemmed=[stemmer.stem(w) for w in stop]
        ############## REMOVE NON ALPHA ################
        alpha = [st for st in stemmed if st.isalpha()]
        ############## JOIN ################
        filtered_arts.append(" ".join(alpha))
    return filtered_arts


In [73]:
filteredArts = pre_process_data(twenty_train.data)

Procesado: 99%

In [74]:
#Se guarda el resultado del preprocesamiento en un archivo mediante la librería Pickle.
import pickle

with open('art_filt.pkl', 'wb') as fp:
    pickle.dump(filteredArts, fp)

In [75]:
#Se lee el resultado del preprocesamiento guardado en un archivo mediante la librería Pickle.
#import pickle
with open('art_filt.pkl','rb') as fp:
    articlesList = pickle.load(fp)

In [76]:
#Se verifica que los artículos hayan sido obtenidos correctamente.
print(str(len(articlesList)))
print(articlesList[0])

11314
i wonder anyon could enlighten car i saw day it sport car look late earli it call bricklin the door realli small in addit front bumper separ rest bodi this i know if anyon tellm model name engin spec year product car make histori whatev info funki look car pleas


### Vectorización
Explicación de Vectorización y de Count Vectorizer
Explicación de min_df y max_df

In [77]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
count_vectorizer = CountVectorizer(max_df = 0.1, min_df = 5)
# tfidf_vectorizer = TfidfVectorizer(max_df = 0.8, min_df = 10)
trainedData = count_vectorizer.fit_transform(articlesList)
trainedDataArray = trainedData.toarray() #sparsed -> expanded matrix
trainedData.shape

(11314, 12594)

In [78]:
# Se imprime el vocabulario resultante en un txt con el fin de analizarlo visualmente.
with open('word_list.txt', 'w') as fp:
    for word in count_vectorizer.get_feature_names():
        fp.write(word + "\n")

### Pandas
Explicación de Pandas y su uso

In [79]:
import pandas as pd
alpha = 1
cols = count_vectorizer.get_feature_names()
df = pd.DataFrame(trainedDataArray, columns = cols)
df["targetCode"] = twenty_train.target
df.head()

,aa,aaa,aamir,aaron,ab,abandon,abbey,abbott,abbrevi,abc,...,zoom,zr,zs,zterm,zu,zubov,zv,zy,zz,targetCode
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14


## Entrenamiento del modelo.
### Naive Bayes
Explicación y fórmulas
#### Probabilidades a priori y a posteriori.
#### Likelihoods y LogLikelihoods
### Smoothing Laplaciano

In [80]:
N_cat = 20
N_arts = df.values.shape[0]
N_words = df.values.shape[1] - 1 #-1 target's columns
prioriProbs = list()
catProbs = list()

for i in range(N_cat):
    catOcurrency = sum(df.loc[df["targetCode"] == i].drop("targetCode", axis=1).values) + alpha #Sum of category ocurrencies
    prioriProbs.append(np.log(catOcurrency/sum(catOcurrency)))
    catProbs.append(np.log(df.loc[df["targetCode"] == i].shape[0]/N_arts))



## Testeo del modelo.
### Métricas
Explicación de métricas y de cual utilizamos

### Prueba con set de Train.

In [81]:
predictionOk = 0
for n_art in range(N_arts):
    maxIdx = -1
    maxLogL = -float('inf')
    for i in range(N_cat):
        logL = np.dot((trainedDataArray[n_art]),prioriProbs[i])+catProbs[i]
        if (logL > maxLogL):
            maxLogL = logL
            maxIdx = i
    if(maxIdx == twenty_train.target[n_art]):
        predictionOk += 1
print("Accuracy Train: " + str(predictionOk/N_arts))

Accuracy Train: 0.8607035531200283


### Prueba con set de Test.

In [82]:
#Preprocesamiento del set de Test.
filteredTestArts = pre_process_data(twenty_test.data)

Procesado: 99%

In [83]:
#Se guarda el resultado del preprocesamiento en un archivo mediante la librería Pickle.
with open('art_filt_test.pkl', 'wb') as fp:
    pickle.dump(filteredTestArts, fp)


In [84]:
#Se lee el resultado del preprocesamiento guardado en un archivo mediante la librería Pickle.
with open('art_filt_test.pkl','rb') as fp:
    testArticlesList = pickle.load(fp)
#Se verifica que los artículos hayan sido obtenidos correctamente.
print(len(testArticlesList))
print(testArticlesList[0])

7532
i littl confus model bonnevill i hear le se lse sse ssei could someon tell differ far featur perform i also curious know book valu prefer model and much less book valu usual get in word much demand time year i hear earli summer best time buy


In [85]:
# Vectorización del set de test.
testData = count_vectorizer.transform(testArticlesList)
testDataArray = testData.toarray()
print(testData.shape)

(7532, 12594)


### Obtención del accuracy del modelo.

In [86]:
predictionOk = 0
for n_art in range(len(twenty_test.target)):
    maxIdx = -1
    maxLogL = -float('inf')
    for i in range(N_cat):
        logL = np.dot((testDataArray[n_art]),prioriProbs[i])+catProbs[i]
        if (logL > maxLogL):
            maxLogL = logL
            maxIdx = i
    if(maxIdx == twenty_test.target[n_art]):
        predictionOk += 1
print("Accuracy Test: " + str(predictionOk/len(twenty_test.target)))

Accuracy Test: 0.7308815719596389
